# Ray Serve Mistral 8x22B Instruct v0.1 Model

This notebook shows how to serve [mistralai/Mixtral-8x22B-Instruct-v0.1](https://huggingface.co/mistralai/Mixtral-8x22B-Instruct-v0.1) model using multi-GPU, multi-node deployment.

## Setup and Imports

In [ ]:
! pip install kubernetes
! pip install boto3

In [ ]:
import os
import subprocess
import sys

# Set working directory
os.chdir(os.path.expanduser('~/amazon-eks-machine-learning-with-terraform-and-kubeflow'))
print(f"Working directory: {os.getcwd()}")

# Get the src directory
src_dir = os.path.join(os.getcwd(), "src")
sys.path.insert(0, src_dir)

from k8s.utils import (
    wait_for_helm_release_pods,
    wait_for_rayservice_ready, 
    find_k8s_service
)

# Get notebook directory
notebook_dir = os.path.join(os.getcwd(), 'examples', 'inference', 'rayserve', 'mixtral-8x22b-instruct-v01-vllm')
print(f"Notebook directory: {notebook_dir}")

# initialize key variables
release_name = 'rayserve-mixtral-8x22b-instruct-v01'
namespace = 'kubeflow-user-example-com'
hf_model_id = 'mistralai/Mixtral-8x22B-Instruct-v0.1'

## Step 1: Build and Push Docker Container

**Note:** This step builds a custom Docker container for Ray Serve. The region is automatically detected from your AWS configuration.

In [ ]:
import sys
import boto3

# Create a Boto3 session
session = boto3.session.Session()

# Access the region_name attribute to get the current region
current_region = session.region_name

cmd = ['./containers/ray-pytorch/build_tools/build_and_push.sh', current_region]

# Start the subprocess with streaming output
process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, 
                          text=True, bufsize=1, universal_newlines=True)

# Stream output line by line
for line in process.stdout:
    print(line, end='')  # end='' prevents double newlines
    sys.stdout.flush()   # Force immediate output

# Wait for the process to complete and get the return code
return_code = process.wait()

if return_code != 0:
    print(f"\nProcess exited with return code: {return_code}")
else:
    print("\nProcess completed successfully")

## Step 2: Download Hugging Face Mistral 8x22B Instruct v0.1 Pre-trained Model Weights

**Note:** Set your Hugging Face Token below.

In [ ]:
# Replace with your actual Hugging Face token
hf_token = None
assert hf_token, "Please provide your Hugging Face token"

cmd = [
    'helm', 'install', '--debug', release_name,
    'charts/machine-learning/model-prep/hf-snapshot',
    '--set-json', f'env=[{{"name":"HF_MODEL_ID","value":"{hf_model_id}"}},{{"name":"HF_TOKEN","value":"{hf_token}"}}]',
    '-n', 'kubeflow-user-example-com'
]

result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr)

In [ ]:
# wait for model download job to complete
wait_for_helm_release_pods(release_name=release_name, namespace=namespace)

In [ ]:
# Uninstall the Helm chart after completion
cmd = ['helm', 'uninstall', release_name, '-n', namespace]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr)

## Step 3: Launch Ray Service

In [ ]:
cmd = [
    'helm', 'install', '--debug', release_name,
    'charts/machine-learning/serving/rayserve/',
    '-f', f'{notebook_dir}/rayservice.yaml',
    '-n', namespace
]

result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr)

In [ ]:
# Wait for RayService to be ready
wait_for_rayservice_ready(release_name=release_name, namespace=namespace)

## Step 4: Check Service Status

In [ ]:
# Check service status
service_name = f"{release_name}-serve-svc"
service = find_k8s_service(service_name, namespace)
if service:
    print(f"Service {service.metadata.name} is available.")
    print(f"Service type: {service.spec.type}")
    print(f"Service ports: {service.spec.ports} ")
else:
    print(f"Service {service_name} not found.") 

## Step 5: Stop Service

When you're done with the service, run this cell to clean up resources.

In [ ]:
cmd = ['helm', 'uninstall', release_name, '-n', namespace]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr)